In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.tree
import seaborn as sns
from scipy import stats

from sklearn.model_selection import train_test_split

In [2]:
data=pd.read_csv('E:\\pydata\\jhjj_PQR\\jhjj_pqr.csv',encoding='gbk')

In [3]:
#data[data['contract_amt']!=data['amt_fix']]

In [4]:
data.columns

Index(['Unnamed: 0', 'data_date', 'CONTRACTNO', 'odd', 'cts', 'bal',
       'bal_corpus', 'contract_id', 'apply_type', 'regist_utm_source',
       'apply_utm_source', 'age', 'sex', 'score', 'main_model_score',
       'model_id', 'apply_time', 'month_apply', 'day_apply', 'mob',
       'score_cut', 'age_cut', 'channel_reg', 'num', 'od30', 'od30_bal',
       'contract_amt'],
      dtype='object')

## 设计score切点值

In [14]:
## 计算score的切值    先合并contract信息
dat_amt_fix=data.loc[:,['contract_id','score','month_apply','apply_type']][(data['mob']==0)&(data['apply_type']=='再贷')]
dat_amt_fix=dat_amt_fix.sort_values('score',ascending=False)
dat_amt_fix=dat_amt_fix.set_index(np.arange(len(dat_amt_fix['contract_id'])))
#dat_amt_fix

## 计算不同月份不同头部客户的分数 
mon_list=list(dat_amt_fix['month_apply'].unique())

rate=0.5
mon=[]
score_cut_p=[]
for i in mon_list:
    dat_mon_cut=dat_amt_fix[(dat_amt_fix['month_apply']==i)]# & (dat_amt_fix['score']>0)]
    dat_mon_cut=dat_mon_cut.sort_values('score',ascending=False)
    dat_mon_cut=dat_mon_cut.set_index(np.arange(len(dat_mon_cut['contract_id'])))
    score_cutpoint=dat_mon_cut.loc[int(len(dat_mon_cut['contract_id'])*rate),'score']
    mon.append(i)
    score_cut_p.append(score_cutpoint)

dat_ind_score_cut=pd.DataFrame({'month_apply':mon,'score_point_cut':score_cut_p})
dat_ind_score_cut

,month_apply,score_point_cut
0,2018-05-01,869.0
1,2018-08-01,909.0
2,2018-06-01,884.0
3,2018-10-01,895.0
4,2018-11-01,893.0
5,2018-07-01,898.0
6,2018-09-01,888.0
7,2018-04-01,851.0
8,2018-03-01,867.0
9,2018-01-01,0.0


In [6]:
## 用随机种子的方式筛选愿意接受提额的客户
#x=data
#y=data['num']
#x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.25, random_state=0)

#print(x_train.shape)
#print(x_test.shape)
#print(y_train.shape)
#rint(y_test.shape)

## 变量整理与新变量生成

In [7]:
data1=pd.merge(data,dat_ind_score_cut)
data=data1

data['amt_fix']=data.apply(lambda hxg:3000 if (hxg['apply_type']=='再贷') & (hxg['score']>hxg['score_point_cut'])
                          else hxg['contract_amt'],axis=1)
#data['bal_fix']=data.apply(lambda hxg:hxg['bal']*hxg['amt_fix']/hxg['contract_amt'],axis=1)

data['bal_fix']=data['bal']*data['amt_fix']/data['contract_amt']
data['od30_bal_fix']=data['bal_fix']*data['od30']

In [8]:
data.columns

Index(['Unnamed: 0', 'data_date', 'CONTRACTNO', 'odd', 'cts', 'bal',
       'bal_corpus', 'contract_id', 'apply_type', 'regist_utm_source',
       'apply_utm_source', 'age', 'sex', 'score', 'main_model_score',
       'model_id', 'apply_time', 'month_apply', 'day_apply', 'mob',
       'score_cut', 'age_cut', 'channel_reg', 'num', 'od30', 'od30_bal',
       'contract_amt', 'score_point_cut', 'amt_fix', 'bal_fix',
       'od30_bal_fix'],
      dtype='object')

## 聚合瘦身

In [11]:
# 聚合
groupby_key=['data_date','odd','cts','apply_type','sex','model_id','month_apply','score_cut','age_cut','channel_reg','mob']
dat_grp=data.groupby(by=groupby_key,as_index=False).agg({'bal_corpus':sum,'num':sum,'od30':sum,'od30_bal':sum,'bal_fix':sum,'contract_amt':sum,'amt_fix':sum,'od30_bal_fix':sum})


In [12]:
## 导出聚合CSV
dat_grp.to_csv('E:\\pydata\\jhjj_3000\\jhjj3000_grp.csv',encoding='gbk')